<a href="https://colab.research.google.com/github/MarcioB1999/Iniciacao_cientifica/blob/main/idcode_H3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from itertools import permutations as pt
import numpy as np
import pandas as pd  
import networkx as nx
import matplotlib.pyplot as plt
from collections import OrderedDict
import seaborn as sns
from matplotlib import transforms


In [ ]:
class vertice:

  def __init__(self, conf, numb):
    self.number=numb

    self.c=conf

    self.ident_A=[]

    self.peso=sum(conf)


#Essa função é responsavel para pegar o identificador de cada grafo individualmente
  def Identificador_c(self):

    #grafo representado como dicionario para facilitar  
    A={0:[0,1,3],1:[0,1,2],2:[1,2,5],3:[0,3,4],4:[3,4,5,7],5:[2,4,5],6:[6,7],7:[4,6,7,8],8:[7,8]}
    #cada nó é seu proprio vizinho pois ele pode pertencer ao seu identificador
    ident_Aux=[]

    #para cada nó "i" do grafo vou ver seus vizinhos, e quem for marcado coloco ele na lista de identificador de "i"
    #ao final todas listas vao ser uma linha da matriz de identificadores, onde cada linha "i" é o identificador do nó "i"
    for i in list(A.keys()):
      
      ident_i=[]
      aux=[]
      
      #percorrendo vizinhos do nó "i" para saber quem esta marcado
      for j in A[i]:
        if(self.c[j]==1):
          ident_i.append(j)


      #adiciono a lista de identificador de "i" na "matriz"
      ident_Aux.append(ident_i)
    

    self.ident_A=ident_Aux



  def validacao(self):
      

      aux1=pd.DataFrame(self.ident_A).iloc[[1,4,7]].copy()
      
      aux2=aux1.copy()

      aux2.drop_duplicates(inplace=True)

      aux2.dropna(inplace=True, how='all') 
     
      if(len(aux2)==len(aux1)):
          return True
      else:
          return False
  

In [ ]:
class aresta:

    def __init__(self, config1, config2):

        self.c=list(np.concatenate((config1.c, config2.c)))
        self.no1=config1
        self.no2=config2
        self.peso=(config1.peso+config2.peso)/2
        self.ident_A=[]


    def Juncao(self):
        #definindo novas arestas ao se juntar os dois grafos
        B={2:15,5:12,8:9,9:8,12:5,15:2}

  
        #aqui vou entrar na matriz de identificador do grafo 2, e mudar seus indices, pois agora ele esta junto com o 1, o indice dos
        #seus vizinhos mudam, olhar imagem dos grafos juntos
        aux = list(pd.DataFrame(self.no2.ident_A).replace({0:9,1:10,2:11,3:12,4:13,5:14,6:15,7:16,8:17,np.nan:'lixo'}).values)
        #ao converter ele para dataframe, ele deixa todas as linhas da matriz com mesmo tamanho, preenchendo com Nan, onde n tinha valores
        #Exemplo: [[1,2],[5]]--->[array([1,2]),array([5.Nan])]
        #e cada linha vai ser do tipo array

        #para arrumar isso, troquei os Nan de cada vetor pela string "lixo", e depois percorri cada linha da matriz e tirei os Nan por consequencia
        #diminuindo a dimensao da linha
        for i in range(len(aux)):
            #convertendo as linhas para lista
            aux[i]=list(aux[i])
            #tirando os Nan
            aux[i]=list(filter(('lixo').__ne__, aux[i]))
        
        ident_Aux=self.no1.ident_A+aux


        #para adicionar um elemento novo ao identificador dos nós que se juntaram ao outro grafo, caso precise
        for i in list(B.keys()):
            #B[i] é o indice do vizinho de i
            #C[B[i]]=se o vizinho de i é marcado ou n
            if(self.c[B[i]]==1):
                #pegar o identificador do i e colocar mais esse vizinho
                if (i<9):
                  ident_Aux[i]=ident_Aux[i]+[B[i]]
                else:
                  ident_Aux[i]=[B[i]]+ident_Aux[i]

        self.ident_A=ident_Aux
    

    def Validacao(self):
        aux1=pd.DataFrame(self.ident_A).iloc[[1,2,4,5,7,8,9,10,12,13,15,16]].copy()
      
        aux2=aux1.copy()
        aux2.drop_duplicates(inplace=True)
        aux2.dropna(inplace=True, how='all') 

        if(len(aux2)==len(aux1)):
            return True
        else:
            return False

In [ ]:
class solver:

    def __init__(self):
        self.n=None
        self.N=None
        self.config_validas=[]
        self.uniao_validas=[]
        self.adjM=[]


    def Constr_Config(self):

        c=np.zeros(9) 
        all=c.copy()

        for i in range(9):
            
            c[i]=1
            perm=pd.DataFrame(pt(c)).drop_duplicates()
            all=np.vstack([all, perm])

        val=[]
        numb=0

        for conf in all:

            i=vertice(conf,numb)
            i.Identificador_c()

            if(i.validacao()):
                val.append(i)
                numb+=1

        self.config_validas=val
        self.n=len(val)
        self.N=2*len(val)

    
    def Constr_Juncao(self):

        validos=[]

        for i in self.config_validas:

            for j in  self.config_validas:

                aux=vertice(j.c,j.number+self.n)
                aux.Identificador_c()

                uniao=aresta(i,aux)
                uniao.Juncao()

                if(uniao.Validacao()):
                    validos.append(uniao)

    
        self.uniao_validas=validos



    def Const_Matriz(self):
        

        A=np.zeros((self.N,self.N))
        for i in range(self.N):
          for j in range(self.N):
            A[i,j] = 10**(10)

        for i in self.uniao_validas:
            A[i.no1.number,i.no2.number]=A[i.no1.number+self.n,i.no2.number-self.n]=i.peso
        
        print(f'valor min na matriz da iteração {iter} = {A.min()}')
        print(f'valor max na matriz da iteração {iter} = {A.max()}')
        print('-----------------------------------------------------------------')


        self.adjM = np.array(A)

       




    def Floyd_Warshall(self):
        dist = self.adjM
        for k in range(self.N):
            for i in range(self.N):
                if (dist[i][i]<0):
                    return False #quando tem ciclo negativo
                for j in range(self.N):
                  if (dist[i][k]<100)and(dist[k][j]<100):
                    dist[i][j] = min(dist[i][j], dist[i][k] + dist[k][j])
        return True #nao tem ciclo negativo
                




In [ ]:
S=solver() 
S.Constr_Config()
print(len(S.config_validas))
#S.Constr_Juncao()


378


In [ ]:
2**9

512

In [ ]:
S.Const_Matriz()

valor min na matriz da iteração <built-in function iter> = 3.0
valor max na matriz da iteração <built-in function iter> = 10000000000.0
-----------------------------------------------------------------


In [ ]:
print(len(S.uniao_validas))

57723


In [ ]:
print(S.adjM[0][755])

6.0


In [ ]:
adjM = S.adjM.copy()
N = S.N
n = S.n
print(n,N)

378 756


In [ ]:
matriz_menor = np.zeros((n,n))
matriz_salto = np.zeros((n,n))
for i in range(n):
    for j in range(n):
        matriz_menor[i][j] = 10**(10)
        matriz_salto[i][j] = int(-1)
        for k in range(n):
            l = k + n
            if (adjM[i][l]>100): continue
            if (adjM[l][j]>100): continue
            if (matriz_menor[i][j]<=adjM[i][l]+adjM[l][j]): continue
            if (S.config_validas[i].c[2]==0)and(S.config_validas[j].c[0]==0)and(S.config_validas[k].c[6]==0)and(S.config_validas[k].c[8]==0): continue
            if (S.config_validas[i].c[5]==0)and(S.config_validas[j].c[3]==0)and(S.config_validas[k].c[0]==0)and(S.config_validas[k].c[2]==0)and(S.config_validas[k].c[3]==0)and(S.config_validas[k].c[5]==0): continue
            if (S.config_validas[i].c[8]==0)and(S.config_validas[j].c[0]==0)and(S.config_validas[k].c[0]==0)and(S.config_validas[k].c[2]==0)and(S.config_validas[k].c[3]==0)and(S.config_validas[k].c[5]==0): continue
            matriz_menor[i][j] = adjM[i][l]+adjM[l][j]
            matriz_salto[i][j] = int(k)



In [ ]:
print(matriz_menor)

[[10.  10.  10.  ... 11.5 11.5 12. ]
 [10.  10.  10.  ... 11.5 11.5 12. ]
 [10.  10.  10.  ... 10.5 10.5 11. ]
 ...
 [ 9.5  8.5  8.5 ... 11.  11.  11.5]
 [ 9.5  8.5  8.5 ... 11.  11.  11.5]
 [10.   9.   9.  ... 11.5 11.5 12. ]]


In [ ]:
print(N)
print(S.n)
print(adjM[0][755])
print(S.adjM[0][755])
print(adjM[51][433])
print(adjM[433][51])
print(adjM[55][429])
print(adjM[429][55])
print(S.config_validas[51].c)
print(S.config_validas[55].c)
for i in S.uniao_validas:
    if (i.no1.number==51)and(i.no2.number==433):
        print(i.no1.number,i.no2.number,i.peso,i.c,i.ident_A)

756
378
6.0
6.0
10000000000.0
4.0
4.0
10000000000.0
[1. 0. 1. 1. 0. 0. 0. 0. 1.]
[1. 0. 1. 0. 0. 1. 1. 0. 0.]


In [ ]:
def rastreio(rast,i,j):
  k = int(rast[i][j])
  if (k==-1): return [j]
  if (k==i): return []
  if (k==j): return []
  #print(i,j,k)
  B = []
  B = B + rastreio(rast,i,k)
  B = B + rastreio(rast,k,j)
  return B


In [ ]:
def Floyd(adjM, N):
    rast = np.zeros((N,N))
    for i in range(N):
      for j in range(N):
        rast[i][j] = -1

    dist = adjM.copy()
    for k in range(N):
        for i in range(N):
            for j in range(N):
              if (dist[i][k]<100)and(dist[k][j]<100):
                if (dist[i][j] > dist[i][k] + dist[k][j]):
                  dist[i][j] = dist[i][k] + dist[k][j]
                  rast[i][j] = k
                if (i==j)and(dist[i][i]<0):
                  print(i,j,k,dist[i][j])
                  A = [i]
                  A = A + rastreio(rast,i,k)
                  A = A + rastreio(rast,k,i)
                  return A #quando tem ciclo negativo
    return [-1,-1] #nao tem ciclo negativo


In [ ]:
#for i in range(N):
#  for j in range(N):
#    adjM[i][j] = S.adjM[i][j] - 4.1

In [ ]:
matriz_menor2 = matriz_menor.copy()
for i in range(n):
  for j in range(n):
    matriz_menor2[i][j] = matriz_menor[i][j] - 2*9*(6/13) + 0.00000001    # 4.1538

In [ ]:
matriz_menor2[105][38] += 0.5  # += 0.5 # para evitar o codigo invalido com 105
matriz_menor2[14][105] += 0.0  # += 0.5 # para evitar o codigo invalido com 105

matriz_menor2[107][20] += 0.5  # += 0.5 # para evitar o codigo invalido com 107
matriz_menor2[88][107] += 0.0  # += 0.5 # para evitar o codigo invalido com 107



In [ ]:
#A = Floyd(adjM, N)
A = Floyd(matriz_menor2, n)
print(A)

179 179 177 -0.0014000000000038426
[179, 88, 117, 51, 53, 62, 97, 123, 54, 55, 65, 38, 177, 179]


In [ ]:
#A = [179, 88, 117, 51, 53, 62, 97, 123, 54, 55, 65, 38, 177, 179] # Melhor codigo valido encontrado - Densidade 108/(26*9) = 6/13 = 0.46154
#A = [177,61,54,55,65,38,177]  # codigo valido muito bom - Densidade 50/(12*9) = 25/54 = 0.463
#A = [179, 88, 117, 51, 53, 62, 14, 105, 38, 177, 179] # codigo invalido por causa do 105
#A = [179, 88, 107, 20, 123, 54, 55, 65, 38, 177, 179] # codigo invalido por causa do 107
B = [A[0]]
for i in range(len(A)-1):
  B = B + [ int(matriz_salto[A[i]][A[i+1]]), A[i+1] ]

print(A)
print(B)

for i in range(len(B)):
  print(S.config_validas[B[i]].c)

erro = 0
for x in range(len(B)-2):
    i = B[x]
    k = B[x+1]
    j = B[x+2]
    if (S.config_validas[i].c[2]==0)and(S.config_validas[j].c[0]==0)and(S.config_validas[k].c[6]==0)and(S.config_validas[k].c[8]==0): erro=1; break
    if (S.config_validas[i].c[5]==0)and(S.config_validas[j].c[3]==0)and(S.config_validas[k].c[0]==0)and(S.config_validas[k].c[2]==0)and(S.config_validas[k].c[3]==0)and(S.config_validas[k].c[5]==0): erro=1; break
    if (S.config_validas[i].c[8]==0)and(S.config_validas[j].c[0]==0)and(S.config_validas[k].c[0]==0)and(S.config_validas[k].c[2]==0)and(S.config_validas[k].c[3]==0)and(S.config_validas[k].c[5]==0): erro=1; break
if (erro==1): print("Codigo Invalido em ",i,k,j)
else: print("CODIGO VALIDO")

[179, 88, 117, 51, 53, 62, 97, 123, 54, 55, 65, 38, 177, 179]
[179, 123, 88, 54, 117, 55, 51, 65, 53, 38, 62, 177, 97, 179, 123, 88, 54, 117, 55, 51, 65, 53, 38, 62, 177, 97, 179]
[1. 0. 1. 0. 1. 0. 0. 1. 1.]
[0. 0. 1. 0. 1. 1. 0. 1. 0.]
[0. 1. 1. 0. 0. 1. 0. 1. 0.]
[1. 0. 1. 0. 1. 0. 0. 0. 1.]
[0. 0. 1. 1. 0. 1. 0. 1. 0.]
[1. 0. 1. 0. 0. 1. 1. 0. 0.]
[1. 0. 1. 1. 0. 0. 0. 0. 1.]
[1. 0. 0. 1. 0. 1. 0. 1. 0.]
[1. 0. 1. 0. 1. 0. 1. 0. 0.]
[1. 1. 0. 1. 0. 0. 0. 1. 0.]
[1. 0. 0. 1. 1. 0. 0. 1. 0.]
[1. 0. 1. 0. 1. 0. 1. 1. 0.]
[0. 1. 0. 1. 0. 1. 0. 1. 0.]
[1. 0. 1. 0. 1. 0. 0. 1. 1.]
[0. 0. 1. 0. 1. 1. 0. 1. 0.]
[0. 1. 1. 0. 0. 1. 0. 1. 0.]
[1. 0. 1. 0. 1. 0. 0. 0. 1.]
[0. 0. 1. 1. 0. 1. 0. 1. 0.]
[1. 0. 1. 0. 0. 1. 1. 0. 0.]
[1. 0. 1. 1. 0. 0. 0. 0. 1.]
[1. 0. 0. 1. 0. 1. 0. 1. 0.]
[1. 0. 1. 0. 1. 0. 1. 0. 0.]
[1. 1. 0. 1. 0. 0. 0. 1. 0.]
[1. 0. 0. 1. 1. 0. 0. 1. 0.]
[1. 0. 1. 0. 1. 0. 1. 1. 0.]
[0. 1. 0. 1. 0. 1. 0. 1. 0.]
[1. 0. 1. 0. 1. 0. 0. 1. 1.]
CODIGO VALIDO
